# Previsão IPCA mês seguinte: Vetores Autoregressivos vs. Machine Learning com conjunto de informação comum

Neste notebook é feito um exercício de comparação da acurácia das previsões de um modelo de vetores autoregressivos com as previsões de diversos modelos de Machine-Learning **usando o mesmo conjunto de informação**. O objetivo do exercício é verificar se, mesmo em um contexto de poucas variáveis explicativas, os modelos ML tem performance melhor que modelos VAR. Além disso, comparamos as previsões usando *expanding window* vs. *rolling window*.  

Todos os dados utilizados foram extraídos do Banco Central através do pacote `sgs` (https://github.com/rafpyprog/pySGS)

O modelo VAR usado é basedo no VAR I (https://www.bcb.gov.br/htms/relinf/port/2012/09/ri201209P.pdf, página 107) do Banco Central e tem as seguintes características:  
- Sem ajuste sazonal  
- Lag 2  
- Deflator do juros: IGP-M (série 189)  
- Variáveis endógenas: Preços livres (série 11428), Preços administrados (série 4449), Taxa de câmbio (série 3696) e Taxa de juros reais = Selic (série 4189) - deflator  

**O modelos de machine learning utilizados são**:
- Boosting
- Random Forest
- SVM
- Regressão Linear

Como o VAR I usa duas defasagens, precisamos dos dados do mês atual e do mês anterior para prever o inflação do mês seguinte. Por esse motivo, construo uma base para os modelos de ML onde, para cada data, os features são o valor contemporâneo das variáveis endógenas utilizadas no VAR I e o valor do mês anterior destas mesmas séries. Obviamente, o target dos modelos será o IPCA no mês seguinte. Ainda, como o número de dados disponíveis não é muito grande, foi decidido não realizar *tunning* dos hiperparâmetros dos modelos para evitar ao máximo problemas de overfitting.  
Por fim, como o ano de 2020 foi marcado pelo grande choque econômico gerado pela pandemia do COVID-19, é feita a comparação para resultados pré início da pandemia e pós início da pandemia.  

Abaixo apresento e analiso os resultados obtidos e, em seguida, exponho o código utilizado para realizar a análise.  
*Portanto o notebook não pode ser replicado de forma linear (observar numeração ao lado das células de códigos)!*

## Resultados e análise:
Tendo rodado o código para o período pré covid e salvando o resultado como `metricas_pré_pandemia` e, posteriormente, rodado para o período pós início da pandemia e salvando o resultado como `metricas_pandemia`, montamos a base de dados do resultado consolidada da seguinte forma:

In [22]:
metricas_pandemia = metricas_pandemia.add_suffix('_pós')
metricas_pandemia.index.name = 'modelo'
metricas_pre_pandemia = metricas_pre_pandemia.add_suffix('_pré')
metricas_pre_pandemia.index.name = 'modelo'
metricas_consolidadas = metricas_pre_pandemia.merge(metricas_pandemia, on='modelo')
metricas_consolidadas

,MSE_pré,Mean Error_pré,MSE_pós,Mean Error_pós
modelo,,,,
var1_forecasts_exp,0.073368,0.058797,0.080728,0.063531
var1_forecasts_rol,0.069442,0.024223,0.078721,0.032849
boost_exp,0.093009,0.061905,0.101697,0.067517
rf_exp,0.087961,0.057357,0.088836,0.063182
svm_exp,0.074371,0.051713,0.084430,0.060095
linreg_exp,0.073410,0.062150,0.080700,0.066620
boost_rol,0.064479,0.013049,0.079808,0.027711
rf_rol,0.061378,0.015668,0.072337,0.030072
svm_rol,0.078578,0.033560,0.083862,0.039358


### Análise dos resultados:

Começando com observações gerais, primeiro nota-se que (exceto para o SVM) todos os modelos apresentaram erro quadrático médio menor nas previsões com *rolling window* do que nas previsões com *expanding window*. Isso pode ser um indício de diferentes regimes para o processo de inflação (para mais sobre modelo de diferentes regimes para inflação ver Amisano e Fagan, 2013). Em segundo lugar, observa-se que todos os modelos apresentam erro médio positivo, mostrando que todos tem tendência em superestimar o IPCA. Uma possível explicação para este fenômeno pode ser dado pelo fato de que o comportamento da inflação a partir da metade da amostra analisada é de queda acentuada. Por fim os modelos tiveram performance pior na amostra que inclui o período da pandemia. Tal resultado era esperado tanto pelo choque externo da pandemia quanto pelo aumento da volatilidade da inflação.  

Agora comparando o erro quadrático médio dos modelos, temos que os 3 melhores resultados para o período pré pandemia foram (em ordem do melhor para o pior): Random Forest com rolling window, Boosting com rolling window e Regressão Linear com rolling window. Para o período pós início da pandemia os 3 melhores foram: Random Forest com rolling window, Regressão Linear com rolling window, VAR com rolling window. Destaca-se, então, a boa performance do modelo de Random Forest com rolling window, corroborando com o resultado apresentado em Medeiros et al, 2019. Além disso, **com o mesmo conjunto de informação**, o modelo VAR consegue se manter no mesmo patamar de performance que modelos ML.  

Por fim, é importante ressaltar que **os modelos de ML não foram utilizados em todo seu potencial**. Primeiro, não foi feito nenhum esforço de *tunning* dos modelos de ML, sendo todos utilizados com seus hiperparâmetros padrões do pacote `scikit-learn`. Segundo, e ainda mais relevante, ao usar o mesmo conjunto de informação que o modelo VAR I abrimos mão da capacidade dos modelos de ML em assimilar um grande número de variáveis. Neste contexto, o resultado de que o modelo RF performa melhor que o modelo VAR I mesmo com poucas variáveis explicativas é bastante forte e demonstra que o uso destes modelos pode não ser necessariamente restrito aos casos de alta dimensão de dados.

## Código
Os resultados foram obtidos rodando o seguinte código:  
**Atenção**: o código foi rodado uma vez para o período pré covid (previsões até jan/2020) e outra para o período pós choque do covid (previsões indo até o fim da amostra). Ou seja, **para replicar o resultado é necessário rodar o código duas vezes**, usando o período desejado e salvando o resultado.

In [1]:
import sgs
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

In [ ]:
##################
### Parâmetros ###
##################

#Janela dos dados a serem baixados (não é o janela do backtest!!!)
start = '01/01/2001' #primeira data dos dados
end = '05/11/2020'   #ultima data dos dados

# Dicionários das séries a serem baixadas
var1_codigos = {'precos_livres': 11428,
        'precos_adm': 4449,
        'cambio': 3696,
        'selic': 4189,
        'deflator':189}

# Parametrôs para o loop:
data_ = date(2006,1,1) # primeiro forecast é para data_ + 1 mês
end_date = date(2019, 12, 1) #ultimo forecast é para end_date + 1 mês

delta = relativedelta(months=1) # pular de mês em mês
delta_5anos = relativedelta(months=60) # rolling window de 5 anos

# Modelos Ml (botar os modelos que quiser rodar aqui):
boost = GradientBoostingRegressor()
rf = RandomForestRegressor()
svm = SVR()
linreg = LinearRegression()

# Dicionário para usar no loop (adicionar os modelos selecionados nesse dicionário)
models = {'boost': boost, 'rf': rf, 'svm': svm, 'linreg': linreg}

# Séries auxilares
var1_forecasts_exp = []
var1_forecasts_rol = []
ipca_real = []
index_data = []


In [3]:
# Pegando dados do SGS do Bacen
ipca = sgs.time_serie(433, start, end) # Para comparar forecasts com o realizado

var1_series = {}

for k, v in var1_codigos.items():
    var1_series[k] = sgs.time_serie(v, start = start, end = end)

In [4]:
# Montar base para o os modelos de ML:
ml_df = pd.DataFrame.from_dict(var1_series)
ml_df['juros_real'] = ml_df['selic'] - ml_df['deflator']
ml_df = ml_df.dropna()
ml_df = ml_df[['precos_livres', 'precos_adm', 'cambio', 'juros_real']]

# Usando lag 1 para ter a mesmo conjunto de informação que o VAR
for col in ml_df:
    name = col + '_lag1'
    ml_df[name] = ml_df[col].shift()
ml_df['target'] = ipca.shift(-1)
ml_df = ml_df.dropna()

# Teste raiz unitária:
for col in ml_df:
    adf = adfuller(ml_df[col])
    if adf[1] > 0.10:
        new_name = col + '_dif'
        ml_df[col] = ml_df[col].diff()
        ml_df = ml_df.rename({col: new_name}, axis = 1)
ml_df = ml_df.dropna()

# Dividindo base em features e target
X = ml_df.iloc[:,:-1]
Y = ml_df['target']

In [5]:
# função para usar no loop ML:
def get_forecast(model, X_train, Y_train, X_test):
    
    model.fit(X_train, Y_train)
    return model.predict(X_test)[0]

Separando os loops do VAR dos loops do ML para facilitar a leitura do código:

In [15]:
###############################
### Rolling Window Loop VAR ###
###############################
data = data_
while data <= end_date:
    index_data.append(data)
    
    precos_livres = var1_series['precos_livres'][start:data]
    precos_adm = var1_series['precos_adm'][start:data]
    
    # Montando o DataFrame
    var1_df = pd.DataFrame.from_dict(var1_series)
    var1_df['juros_real'] = var1_df['selic'] - var1_df['deflator']
    var1_df = var1_df.dropna()
    var1_df = var1_df[['precos_livres', 'precos_adm', 'cambio', 'juros_real']]
    var1_df = var1_df[start:data]

    # Teste raiz unitária e botando variáveis com raiz unitária em diferenças
    for col in var1_df:
        adf = adfuller(var1_df[col])
        if adf[1] > 0.10:
            new_name = col + '_dif' # renomeando caso a variável seja transformada para diferenças    
            var1_df[col] = var1_df[col].diff()
            var1_df = var1_df.rename({col: new_name}, axis = 1)
    var1_df = var1_df.dropna()
    
    # Rodando o modelo
    var1_model = VAR(var1_df)
    var1_resultado = var1_model.fit(2) #usando o Lag apontado pelo BACEN
    
    ## Calculando a previsão do IPCA para o mês seguinte
    #1) pegando o peso dos preços adm:
    p_serie = (ipca[:data] - precos_livres)/(precos_adm - precos_livres)
    p = p_serie[-1]

    # Evitar problemas nos dados. Tem data em que p = 0 ou p = NaN! Nestes casos usar dado da data anterior
    if p < 0.05 or str(p) == 'nan':
        p = p_serie[-2]


    #2) Calculando o forecast (se tiver em diferenças somar o forecast ao valor anterior)
    forecast = var1_resultado.forecast(var1_df.values[-2:],1)[0]

    #Preços Livres
    if 'dif' in var1_df.iloc[:,0].name:
        var1_forecast_precos_livres = forecast[0] + precos_livres[-1]
    else:
        var1_forecast_precos_livres = forecast[0]
    
    #Preços administrados
    if 'dif' in var1_df.iloc[:,1].name:
        var1_forecast_precos_adm = forecast[1] + precos_adm[-1]
    else:
        var1_forecast_precos_adm = forecast[1]
    
    #IPCA
    var1_forecast_ipca = (1-p)*var1_forecast_precos_livres + p*var1_forecast_precos_adm
    var1_forecasts_exp.append(var1_forecast_ipca)
    
    
    # Próxima Data!
    data += delta
    ipca_real.append(ipca[data])
    
    
#################################
### Expanding Window Loop VAR ###
#################################
data = data_
while data <= end_date:
    start = data - delta_5anos
    precos_livres = var1_series['precos_livres'][start:data]
    precos_adm = var1_series['precos_adm'][start:data]
    
    # Montando o DataFrame
    var1_df = pd.DataFrame.from_dict(var1_series)
    var1_df['juros_real'] = var1_df['selic'] - var1_df['deflator']
    var1_df = var1_df.dropna()
    var1_df = var1_df[['precos_livres', 'precos_adm', 'cambio', 'juros_real']]
    var1_df = var1_df[start:data]

    # Teste raiz unitária e botando variáveis com raiz unitária em diferenças
    for col in var1_df:
        adf = adfuller(var1_df[col])
        if adf[1] > 0.10:
            new_name = col + '_dif'
            var1_df[col] = var1_df[col].diff()
            var1_df = var1_df.rename({col: new_name}, axis = 1)
    var1_df = var1_df.dropna()
    
    # Rodando o modelo
    var1_model = VAR(var1_df)
    var1_resultado = var1_model.fit(2) #usando o Lag apontado pelo BACEN
    
    ## Calculando a previsão do IPCA para o mês seguinte
    #1) pegando o peso dos preços adm:
    p_serie = (ipca[:data] - precos_livres)/(precos_adm - precos_livres)
    p = p_serie[-1]

    # Evitar problemas nos dados. Tem data em que p = 0 ou p = NaN! Nestes casos usar dado da data anterior
    if p < 0.05 or str(p) == 'nan':
        p = p_serie[-2]


    #2) Calculando o forecast (se tiver em diferenças somar o forecast ao valor anterior)
    forecast = var1_resultado.forecast(var1_df.values[-2:],1)[0]

    #Preços Livres:
    if 'dif' in var1_df.iloc[:,0].name:
        var1_forecast_precos_livres = forecast[0] + precos_livres[-1]
    else:
        var1_forecast_precos_livres = forecast[0]
    
    #Preços administrados:
    if 'dif' in var1_df.iloc[:,1].name:
        var1_forecast_precos_adm = forecast[1] + precos_adm[-1]
    else:
        var1_forecast_precos_adm = forecast[1]
    
    #IPCA
    var1_forecast_ipca = (1-p)*var1_forecast_precos_livres + p*var1_forecast_precos_adm
    var1_forecasts_rol.append(var1_forecast_ipca)
    
    
    # Próxima Data!
    data += delta
    
var1_for_resultados = pd.DataFrame([index_data, ipca_real, var1_forecasts_exp, var1_forecasts_rol]).transpose()
var1_for_resultados.columns = ['Data', 'IPCA', 'var1_forecasts_exp', 'var1_forecasts_rol']
var1_for_resultados = var1_for_resultados.set_index('Data')

In [16]:
start = '01/01/2001' #Reiniciar o start pq rolling window loop do var altera
index_data = [] #Para poder rodar o Loop independete do loop do var
#######################################
### Expanding Window Loop ML Models ###
#######################################
exp = [] # lista para salvar os forecasts 
data = data_ 
while data <= end_date:
    index_data.append(data)
    
    # Selecionando a subamostra
    X_ = X[start:data]
    Y_ = Y[start:data]
    X_train = X_[:-1]
    Y_train = Y_[:-1]
    X_test = X_[-1:]
    
    # Rodando os modelos
    lista_aux = []
    for model in models.values():
            lista_aux.append(get_forecast(model, X_train, Y_train, X_test))
    exp.append(lista_aux)    
    
    data = data + delta

# Montando df
exp = pd.DataFrame(exp, index = index_data, columns = models.keys())
exp = exp.add_suffix('_exp')
exp.index.name = 'Data'

#####################################
### Rolling Window Loop ML Models ###
#####################################
rol = []
data = data_
while data <= end_date:
    start = data - delta_5anos
    
    # Selecionando a subamostra
    X_ = X[start:data]
    Y_ = Y[start:data]
    X_train = X_[:-1]
    Y_train = Y_[:-1]
    X_test = X_[-1:]

    # Rodando os modelos
    lista_aux = []
    for model in models.values():
            lista_aux.append(get_forecast(model, X_train, Y_train, X_test))
    rol.append(lista_aux)    
    data = data + delta

# Montando df
rol = pd.DataFrame(rol, index = index_data, columns = models.keys())
rol = rol.add_suffix('_rol')
rol.index.name = 'Data'

# Juntando os dois df
forecasts_ml = exp.merge(rol, on = 'Data')

In [17]:
# juntando a base d forecasts do var com a base de forecasts dos modelos ML
resultado = var1_for_resultados.merge(forecasts_ml, on = 'Data')

In [18]:
# Calculando MSE e erro médio e juntando um um df
metricas = pd.DataFrame(index = resultado.columns[1:])
mse_list = []
mean_error_list = []
for col in resultado:
    mse = np.sum((resultado['IPCA'] - resultado[col])**2)/resultado.shape[0]
    mean_error = np.mean(resultado[col] - resultado['IPCA'])
    if mse > 0: # para pular a coluna do IPCA
        mse_list.append(mse)
        mean_error_list.append(mean_error)
metricas['MSE'] = mse_list
metricas['Mean Error'] = mean_error_list

## Referências:
**Amisano, G. and G. Fagan, 2013**. Money growth and inflation: a regime switching approach,” Journal of International Money and Finance, 33, 118-145

**Medeiros, M. C., Vasconcelos, G. F., Veiga, Á., and Zilberman, E., 2019**. Forecasting Inflation
in a Data-Rich Environment: The Benefits of Machine Learning Methods. Journal of Business
and Economic Statistics, 0, 1–45
